In [1]:
from source.utils import *
net = load_net('vit_small')

In [2]:
assert torch.cuda.is_available(), "Notebook is not configured properly!"
device = "cuda:0"
checkpoint = "/work/h2020deciderficarra_shared/rxrx1/checkpoints/dino_small_testing/checkpoint0082.pth"
load_weights(checkpoint, net, device)

Take key student in provided checkpoint dict
Pretrained weights found at /work/h2020deciderficarra_shared/rxrx1/checkpoints/dino_small_testing/checkpoint0082.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


{'cls_token': tensor([[[-2.8367e-03,  6.9717e-03,  3.8978e-03,  1.0914e-03, -8.1534e-04,
            2.3986e-03,  3.8291e-03, -1.0737e-02, -4.6385e-03, -4.8687e-04,
           -4.8281e-05, -6.2284e-03,  6.5869e-03, -6.8767e-03,  5.4605e-03,
            2.6760e-03, -2.9807e-03, -3.5134e-03,  1.6805e-03,  6.6187e-04,
            4.6205e-03,  2.8156e-03, -3.8178e-04,  1.0232e-04, -7.9036e-03,
           -6.0880e-03, -6.3049e-04,  9.1913e-03,  1.7433e-03, -1.1573e-03,
            2.3842e-03, -1.5856e-03, -1.8389e-02,  1.5125e-03, -7.4799e-04,
            2.2330e-02,  1.5702e-03,  5.2661e-04, -3.5222e-04, -4.3240e-03,
            4.2376e-03,  3.6031e-03,  5.6568e-03,  1.3837e-03,  3.1107e-03,
           -2.0273e-03, -5.6740e-03,  1.7441e-03, -3.4377e-03, -1.3206e-03,
           -4.9078e-03,  1.7530e-03, -9.9174e-03, -6.3787e-03,  4.1220e-03,
            6.2519e-03,  9.4534e-04,  1.1082e-02,  2.0404e-03,  1.1149e-05,
           -8.2144e-03,  5.1078e-03, -1.7503e-03, -4.9496e-03, -8.0305e-04,

In [3]:
from source.dataset import Rxrx1
from torch.utils.data import Subset
import torchvision.transforms.v2 as transforms
from torch.utils.data import DataLoader
dataset = Rxrx1("/work/ai4bio2024/rxrx1",'metadata_plate_norm_3c.csv')
metadata = dataset.get_metadata()
train_indices = metadata.index[metadata.iloc[:, 3] == 'train'].tolist()
val_indices = metadata.index[metadata.iloc[:, 3] == 'val'].tolist()
test_indices = metadata.index[metadata.iloc[:, 3] == 'test'].tolist()
train_dataset = Subset(dataset, train_indices[:2000])

def dino_test_collate_old(batch):
    """
    Transformations used for dino training that must be used also 
    during validation and head training. For now we are just
    normalizing using ImageNet values of mean and standard deviation.
    A little bit of code is repeated here for the sake of testing.
    """
    image_to_tensor = transforms.Compose([
        transforms.ToImage(), 
        transforms.ToDtype(torch.float),
        transforms.CenterCrop(224)
        ])
    images, sirna_ids, metadata = zip(*batch)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    norm_images = []
    for i, image in enumerate(images):
        variance = metadata[i][12]
        image = image_to_tensor(image)
        image = (image - mean)/std
        norm_images.append(image)
    norm_images = torch.stack(norm_images)         
    return norm_images, sirna_ids, metadata

In [ ]:
batch_size = 20
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                      num_workers=4, drop_last=True, prefetch_factor=2, persistent_workers=True,collate_fn=dino_test_collate_old)
embeddings = []
plates = [] # 5
experiments = [] # 4
cell_type = [] # 2
n = len(train_dataloader)
for i, (x_batch, siRNA_batch, metadata) in enumerate(train_dataloader):
	print(f"{i}/{n}")
	embeddings.append(net(x_batch))
	for sample in metadata:
		cell_type.append(sample[2])
		experiments.append(sample[4])
		plates.append(sample[5])
embs = torch.cat(embeddings, dim=0)
print(embs.shape)

	

In [27]:
from sklearn.decomposition import PCA
x = embs.detach().cpu().numpy().astype('float64')
pca = PCA(n_components=2)
x_reduced = pca.fit_transform(x)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.scatter(x_reduced[:, 0], x_reduced[:, 1], alpha=0.5, s=10)  # alpha for transparency, s for size
plt.xlabel("f1")
plt.ylabel("f2")
plt.title("Scatter Plot of 1000 Samples")

In [29]:
import pandas as pd
data = pd.DataFrame({
    "f1": x_reduced[:, 0],  # First column from the tensor
    "f2": x_reduced[:, 1],  # Second column from the tensor
    "CT": cell_type,
    "E": experiments,
    "P": plates,
})


In [ ]:
import seaborn as sb
sb.scatterplot(data=data, x="f1", y="f2", hue="P")
